In [ ]:
#loading data


In [1]:
import dask.bag as db
import json
stocks = db.read_text("/Users/achyuth/newDocuments/ADT/Json_dataset/part-00000-7597fb35-c887-441a-8fe5-f5f765ba6a25-c000.json")

In [4]:
import json
js = stocks.map(json.loads)

In [9]:
df1 = lazy_df1.compute()

In [113]:
sample_df=df1[1:100000]

In [114]:
import contractions
from keybert import KeyBERT
from textblob import TextBlob
from nltk import sent_tokenize

In [116]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/achyuth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [117]:
import contractions
from keybert import KeyBERT
from textblob import TextBlob
from nltk import sent_tokenize

kw_model = KeyBERT()
def SentimentAnalysis(lines, pos = 0.3, neg = -0.3):
    blob = TextBlob(lines)
    polarity = []
    for sentence in blob.sentences:
        pol=sentence.sentiment.polarity
        if pol >= pos:
            polarity.append('1')
        elif pol <= neg:
            polarity.append('-1')
        else:
            polarity.append('0')
    return polarity

In [119]:
sample_df["Postive_key_words"] =""
sample_df["Negative_key_words"]=""
sample_df["Neutral_key_words"]=""

/var/folders/q4/8hxfb0hn2fb4qx0gwrhjbrwc0000gn/T/ipykernel_11406/1517759878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df["Postive_key_words"] =""
/var/folders/q4/8hxfb0hn2fb4qx0gwrhjbrwc0000gn/T/ipykernel_11406/1517759878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df["Negative_key_words"]=""
/var/folders/q4/8hxfb0hn2fb4qx0gwrhjbrwc0000gn/T/ipykernel_11406/1517759878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [122]:



#with open('Shampoo_Review.txt') as f:
#    lines = f.read()

for num,row in sample_df.iterrows():
    _dict = {'Positive': [], 'Negative': [], 'Neutral': []}
    polarity = SentimentAnalysis(row["reviewText"])
    print(polarity)
    sentences = sent_tokenize(row["reviewText"])
    for i in range(len(sentences)):
        sentence = contractions.fix(sentences[i]) # Expanding words
        keywords = kw_model.extract_keywords(sentence, keyphrase_ngram_range=(2, 2))
        if len(keywords) == 0:
            continue
        print(polarity[i])
        for keyword in keywords:
            if polarity[i] == '1':
                _dict['Positive'].append(keyword[0])
                print(keyword[0])
            elif polarity[i] == '-1':
                _dict['Negative'].append(keyword[0])
                print(keyword[0])
            elif polarity[i] == '0':
                _dict['Neutral'].append(keyword[0])
                print(keyword[0])
    print("len is")
    
    
    
    if len(_dict['Positive'])!=0:
        print("clm number")
        print(num)
        sample_df.at[num,"Postive_key_words"]=list(_dict['Positive'])
    if len(_dict['Negative'])!=0:
        sample_df.at[num,"Negative_key_words"]=list(_dict['Negative'])
    if len(_dict['Neutral'])!=0:
        sample_df.at[num,"Neutral_key_words"]=list(_dict['Neutral'])
        

['0', '0', '0']
0
plastic bottle
expensive plastic
bottle better
better value
bit expensive
0
product performs
performs advertised
0
response supplier
fast response
len is
['1', '0']
1
great product
product love
len is
clm number
2
['0', '0', '1', '0', '1', '0', '1']
0
corner shelf
sturdy floating
floating corner
0
cable box
tv wall
mounted tv
shelf cable
needed shelf
1
product perfect
perfect needs
0
wall shelf
run wall
corner wires
wires run
shelf issue
1
looks great
0
enclosed instructions
instructions helpful
stars enclosed
giving stars
1
good handyman
handyman type
thankfully husband
husband good
type things
len is
clm number
3
['0', '1', '0', '0', '1', '0', '0', '0', '0', '0']
0
purchased shelves
1
look great
0
thought wood
0
read thoroughly
did read
guess did
1
size looking
nice size
look real
did matter
real nice
0
stars shelves
gave stars
shelves coming
apart end
coming apart
0
used months
0
30 lbs
way 30
0
minor issue
great issues
issue just
just bad
bad great
0
glue clamp
tr

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>

In [123]:
import pandas as pd
new_df = pd.DataFrame(columns=["asin","Postive_key_words","Negative_key_words","Neutral_key_words"])


In [124]:
import itertools
row_num=0
def append_to_new_df(_id):
    count=len(new_df)
    row_num=count+1
    new_df.at[row_num,"asin"]=_id
    Postive_key_words_list=list(sample_df[sample_df["asin"]==_id]["Postive_key_words"])
    Negative_key_words_list=list(sample_df[sample_df["asin"]==_id]["Negative_key_words"])
    Neutral_key_words_list=list(sample_df[sample_df["asin"]==_id]["Neutral_key_words"])
    new_df.at[row_num,"Postive_key_words"]=list(itertools.chain.from_iterable(Postive_key_words_list))
    new_df.at[row_num,"Negative_key_words"]=list(itertools.chain.from_iterable(Negative_key_words_list))
    new_df.at[row_num,"Neutral_key_words"]=list(itertools.chain.from_iterable(Neutral_key_words_list))
    
    

In [125]:
asin_list=sample_df["asin"].unique()

In [126]:
for _id in asin_list:
    append_to_new_df(_id)

In [129]:
product_df=sample_df[["asin","brand","main_cat","description","title"]]

In [131]:
product_df=product_df.drop_duplicates(subset=['asin'])

In [132]:
product_df.reset_index(drop=True)

,asin,brand,main_cat,description,title
0,0006564224,Artistic Churchware,Amazon Home,"[16 oz squeeze bottle, 1 lb.]",Artistic Churchware Communion Cup Filler: RW525
1,0560467893,WELLAND,Amazon Home,[Chicago Floating Shelf is an impressively sea...,"WELLAND Chicago Wall Floating Corner Shelf, 20..."
2,0681795107,Timolino,Amazon Home,[Signature Travel Mug is double wall insulated...,Stainless Coffee Mug
3,0752873016,imusti,Amazon Home,[Hardcover storybook.],Asterix and the Falling Sky
4,0768205921,Judy Instructo,Toys & Games,[<DIV>Our geared Judy Clock is now miniature! ...,Judy Instructo My Own Little Judy Clock with B...
...,...,...,...,...,...
234,B00004R938,Melitta,Amazon Home,[Melitta MEX-KITB Espresso Maker with 20-Piece...,Melitta MEXKITB Espresso Maker with 20-Piece Kit
235,B00004R93D,Looney Tunes,Amazon Home,[You can bring the state fair into your home w...,Looney Tunes CCILTA Cotton Candy Maker
236,B00004R940,George Foreman,Amazon Home,[George Foreman GR20WHT 4 Burger Grill Non Sti...,George Foreman GR20WHT XL Grill
237,B00004R941,George Foreman,Amazon Home,[Everybody loves favorite foods from the grill...,George Foreman GR30CB XXL Grill


In [133]:
inner_join_df= pd.merge(new_df, product_df, on='asin', how='inner')
inner_join_df 

,asin,Postive_key_words,Negative_key_words,Neutral_key_words,brand,main_cat,description,title
0,0006564224,[],[],"[plastic bottle, expensive plastic, bottle bet...",Artistic Churchware,Amazon Home,"[16 oz squeeze bottle, 1 lb.]",Artistic Churchware Communion Cup Filler: RW525
1,0560467893,"[great product, product love, product perfect,...","[instructions wrong, wrong sturdy]","[corner shelf, sturdy floating, floating corne...",WELLAND,Amazon Home,[Chicago Floating Shelf is an impressively sea...,"WELLAND Chicago Wall Floating Corner Shelf, 20..."
2,0681795107,"[order mug, mug keeps, tea hot, place order, k...","[new mug, mug quite, quite disappointed, day n...","[mug comes, variety colors, colors hard, comes...",Timolino,Amazon Home,[Signature Travel Mug is double wall insulated...,Stainless Coffee Mug
3,0752873016,"[ufo robots, udeerzo tried, fly galactic, clon...","[worst book, book series, series date, single ...","[end series, asked end, book asked, finished r...",imusti,Amazon Home,[Hardcover storybook.],Asterix and the Falling Sky
4,0768205921,"[tell time, children tell, learning tool, teac...",[],"[numbers printed, including seconds, printed c...",Judy Instructo,Toys & Games,[<DIV>Our geared Judy Clock is now miniature! ...,Judy Instructo My Own Little Judy Clock with B...
...,...,...,...,...,...,...,...,...
234,B00004R938,[],[],[],Melitta,Amazon Home,[Melitta MEX-KITB Espresso Maker with 20-Piece...,Melitta MEXKITB Espresso Maker with 20-Piece Kit
235,B00004R93D,[],[],[],Looney Tunes,Amazon Home,[You can bring the state fair into your home w...,Looney Tunes CCILTA Cotton Candy Maker
236,B00004R940,[],[],[],George Foreman,Amazon Home,[George Foreman GR20WHT 4 Burger Grill Non Sti...,George Foreman GR20WHT XL Grill
237,B00004R941,[],[],[],George Foreman,Amazon Home,[Everybody loves favorite foods from the grill...,George Foreman GR30CB XXL Grill


In [139]:
len(inner_join_df)

239

In [135]:
inner_join_df["Postive_key_words"][2]

['order mug',
 'mug keeps',
 'tea hot',
 'place order',
 'keeps tea',
 'car cup',
 'flat lid',
 'cup holder',
 'hole lid',
 'lid does',
 'cover clippable',
 'version mug',
 'sliding cover',
 'mug just',
 'make sliding',
 'order mug',
 'mug keeps',
 'tea hot',
 'place order',
 'keeps tea',
 'car cup',
 'flat lid',
 'cup holder',
 'hole lid',
 'lid does',
 'cover clippable',
 'version mug',
 'sliding cover',
 'mug just',
 'make sliding',
 'travel mug',
 'mug used',
 'best travel',
 'hands best',
 'use driving',
 'joy use',
 'goes day',
 'day joy',
 'clean coffee',
 'coffee residue',
 'clip lid',
 'removeable clean',
 'lid removeable',
 'coffee mug',
 'great coffee',
 'mug great',
 'chai tea',
 'tea hot',
 'keeps chai',
 'great size',
 'great gift',
 'gift provided',
 'plastic mug',
 'mug types',
 'better plastic',
 'keeps coffee',
 'coffee warm',
 'coffee preferred',
 'drinking temperature',
 'hours coldest',
 'able coffee',
 'temperature hours',
 'coffee warm',
 'mug barely',
 'warm hou

In [136]:
#tx = graph.begin()
#for index, row in inner_join_df.iterrows():
#    tx.evaluate('''
#       MERGE (a:main_cat {name:$category_name})
#       MERGE (b:brand_name {brand_name:$b_name})
#       MERGE (c:product {_id:$p_id,_name:$p_name})
       
#       MERGE (d:Postive_key_words{_word_list:$p_word_list})
#       MERGE (e:Negative_key_words{_word_list:$ne_word_list})
#       MERGE (f:Neutral_key_words{_word_list:$nu_word_list})
#       MERGE (a)-[r:category_brand]->(b)-[r1:brand_product]->(c)
#       MERGE (c)-[r2:positive_attributes]-(d)
#       MERGE (c)-[r3:positive_attributes]-(e)
#       MERGE (c)-[r4:positive_attributes]-(f)
#       ''', parameters = {'category_name': str(row['main_cat']),'b_name': str(row['brand']), 
#                          'p_id': str(row['asin']),'p_name':str(row["title"]),'p_word_list':str(row["Postive_key_words"]),
#                         'ne_word_list':str(row['Negative_key_words']),'nu_word_list':str(row['Neutral_key_words'])})
#graph.commit(tx)

In [140]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user="neo4j", password="neo4j123")
tx = graph.begin()
for index, row in inner_join_df.iterrows():
    tx.evaluate('''
       MERGE (a:main_cat {name:$category_name})
       MERGE (b:brand_name {brand_name:$b_name})
       MERGE (c:product {_id:$p_id,_name:$p_name})
       MERGE (a)-[r:category_brand]->(b)-[r1:brand_product]->(c)
       ''', parameters = {'category_name': str(row['main_cat']),'b_name': str(row['brand']), 
                          'p_id': str(row['asin']),'p_name':str(row["title"])})
graph.commit(tx)

In [141]:
tx = graph.begin()
for index, row in inner_join_df.iterrows():
    if len(row["Postive_key_words"])>=1:
        for i in row["Postive_key_words"]:
            tx.evaluate('''
        MATCH (n {_id: $p_id})
        MERGE (b:Postive_key_words {_word_list:$p_word_list})
        MERGE (n)-[r:positive_attributes]->(b)
        ''',parameters={'p_id':str(row["asin"]),'p_word_list':str(i)})
    if len(row["Negative_key_words"])>=1:
        for i in row["Negative_key_words"]:
            tx.evaluate('''
        MATCH (n {_id: $p_id})
        MERGE (b:Negative_key_words {_word_list:$ne_word_list})
        MERGE (n)-[r:Negative_attributes]->(b)
        ''',parameters={'p_id':str(row["asin"]),'ne_word_list':str(i)})
    if len(row["Neutral_key_words"])>=1:
        for i in row["Neutral_key_words"]:
            tx.evaluate('''
        MATCH (n {_id: $p_id})
        MERGE (b:Neutral_key_words {_word_list:$nu_word_list})
        MERGE (n)-[r:Neutral_attributes]->(b)
        ''',parameters={'p_id':str(row["asin"]),'nu_word_list':str(i)})
graph.commit(tx)

In [142]:
>>> import datetime
>>> now = datetime.datetime.now()
>>> now

datetime.datetime(2022, 7, 19, 10, 12, 10, 404802)

In [146]:



def fetch_related_product(key_word):
    tx = graph.begin()
    result=tx.evaluate('''
    MATCH (n:product)-[r:positive_attributes]-(b:Postive_key_words{_word_list:$key_word})
    return n._name,n._id''',parameters={'key_word':key_word})
    return result
    graph.commit(tx)

    

In [148]:
import time

start = time.time()
print(fetch_related_product("cup holder"))
end = time.time()
print(end - start)

Stainless Coffee Mug
0.10444474220275879


In [145]:
inner_join_df.to_csv("product_attibutes.csv")

In [149]:
len(inner_join_df)

239